User interface to implement 2 deep learning models by using the webcam 
---
This code implements the 2 deep learning models that are both trained to recognize human emotion by analyzing an image of the human face with the difference that one has been trained with Transfer Learning and the other is trained from scratch

---
Class: Machine Learning

By: Sebastian Russo, Guillermo Trigo, Javier Peres

---

Import necessary libraries
---

In [ ]:
#Import necessary libraries
import tensorflow as tf           #Open source library for AI, ML and DL
#from tensorflow import image      #For image processing and manipulation
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.densenet import preprocess_input
import numpy as np                #For numpy arrays and for scientific computing in Python
import cv2                        #OpenCV to perform operations on images and videos with the webcam 
import os
import sys

---

Hyperparameters and global variables
---

In [ ]:
#Add parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from configuration.config_invoke import json_data
#--------------------------------------------------------------------------------------------------------

#Constants and general variables
#Fixed images measurements
img_height = json_data["images"]["height"]
img_width = json_data["images"]["width"]

#7 classes for seven emotions and of course 2 arrays with the labels and actual emojis
class_labels  = json_data["classes"]["labels"]
class_label_emojis = json_data["classes"]["emojis"]

directory_name = json_data["dl_model"]["directory"]
no_transfer_learning_name = json_data["dl_model"]["no_transfer_learning"]["name"]
transfer_learning_name = json_data["dl_model"]["transfer_learning"]["name"]

#Load the pre-trained models into the environment
model_no_transfer_learning = tf.keras.models.load_model(
    f'{directory_name}/{no_transfer_learning_name}') 
model_transfer_learning = tf.keras.models.load_model(
    f'{directory_name}/{transfer_learning_name}')   


---

Functions to preprocess images and make predictions
---
Must be the exact same preprocessing as the one used during model training

In [ ]:
#Function to preprocess a single image
def pre_process_single_image(img, target_size):
    img=cv2.resize(img, target_size)  #Resize image to target size
    
    #img_array=image.img_to_array(img) #Convert the image to an array format
    img_array = img_to_array(img)
    
    #Preprocess array with denseNet preprocessing function
    img_array= preprocess_input(img_array)     
    img_array = img_array * (1. / 255) #Rescale pixels to range [0,1]
    img_array = np.expand_dims(img_array, axis=0)   #Expand dimensions to add batch
    
    return img_array #Return the preprocessed image array

#-------------------------------------------------------------------------------------------

#Function to predict using the selected model
def predict_emotion(model, preprocessed_img):
    prediction = model.predict(preprocessed_img)        #Generate prediction with selected model
    predicted_class = np.argmax(prediction, axis=1)[0]  #Find index of most probable class 
    predicted_label = class_labels[predicted_class]     #Retrieve label corresponding to the predicted class index
    
    return predicted_label                              #Return the predicted label

---

Initialize CV2 for video capture
---

In [ ]:
#Initialize video capture object
cap = cv2.VideoCapture(0)

# Correct way to define codec
four_cc = cv2.VideoWriter_fourcc(*'XVID')
# Create the video writer object
out = cv2.VideoWriter('captured_faces_video.avi', four_cc, 20.0, (640, 480))

#Load the pre-trained face detector from OpenCV
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

Execute window and use selected model
---

In [ ]:
#Prompt user to choose a model
print("Choose a model:")
print("1. Model without Transfer Learning")
print("2. Model with Transfer Learning")
choice = input("Enter your choice (1 or 2): ")

if choice == '1':
    selected_model = model_no_transfer_learning
    print("Using Model without Transfer Learning")
elif choice == '2':
    selected_model = model_transfer_learning
    print("Using Model with Transfer Learning")
else:
    print("Invalid choice. Using Model without Transfer Learning by default.")
    selected_model = model_no_transfer_learning

#-------------------------------------------------------------------------------------------

while True:
    ret, frame = cap.read()
    if not ret:
        break

    #Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop through detected faces
    for (x, y, w, h) in faces:
        #Extract the region of interest (ROI)
        roi_color = frame[y:y + h, x:x + w]

        #Preprocess the face image
        preprocessed_img = pre_process_single_image(roi_color, target_size=(img_height, img_width))

        #Make a prediction using the selected model
        predicted_label = predict_emotion(selected_model, preprocessed_img)

        #Display the label and bounding box on the frame
        label = f'{predicted_label}'
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    #Write the frame with predictions to the video file
    out.write(frame)

    #Display the resulting frame
    cv2.imshow('Video', frame)

    #Check for key press
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        #Save the image with the face box
        cv2.imwrite('captured_face_image_with_label.jpg', frame)
        print(
            "Image captured and saved as 'captured_face_image_with_label.jpg'")

    elif key == ord('q'):
        break

#Release video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()